In [1]:
import face_recognition
import os
import cv2

In [2]:
print("hi")

hi


In [5]:
KNOWN_FACES_DIR = 'known_faces'
UNKNOWN_FACES_DIR = 'unknown_faces'
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2

In [6]:
print('Loading known faces...')
known_faces = []
known_names = []

Loading known faces...


In [14]:
c=KNOWN_FACES_DIR+"/"+name
c

'known_faces/tavva'

In [16]:
for name in os.listdir(KNOWN_FACES_DIR):

    # Next we load every file of faces of known person
    for filename in os.listdir(c):
        print(filename)

.ipynb_checkpoints
IMG_-s4wohp.jpg
IMG_20200113_201328.jpg
IMG_20200115_141540.jpg
Snapchat-633696397.jpg
.ipynb_checkpoints
IMG_-s4wohp.jpg
IMG_20200113_201328.jpg
IMG_20200115_141540.jpg
Snapchat-633696397.jpg


In [22]:
for name in os.listdir(KNOWN_FACES_DIR):

    # Next we load every file of faces of known person
    for filename in os.listdir(name):

        # Load an image
        image = face_recognition.load_image_file(c+"/"+filename)

        # Get 128-dimension face encoding
        # Always returns a list of found faces, for this purpose we take first face only (assuming one face per image as you can't be twice on one image)
        encoding = face_recognition.face_encodings(image)

        # Append encodings and name
        known_faces.append(encoding)
        known_names.append(name)

PermissionError: [Errno 13] Permission denied: 'known_faces/tavva/.ipynb_checkpoints'

In [25]:
for name in os.listdir(KNOWN_FACES_DIR):

    # Next we load every file of faces of known person
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):

        # Load an image
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')

        # Get 128-dimension face encoding
        # Always returns a list of found faces, for this purpose we take first face only (assuming one face per image as you can't be twice on one image)
        encoding = face_recognition.face_encodings(image)[0]

        # Append encodings and name
        known_faces.append(encoding)
        known_names.append(name)

PermissionError: [Errno 13] Permission denied: 'known_faces/tavva/.ipynb_checkpoints'

In [4]:
print(known_faces[0])

NameError: name 'known_faces' is not defined

In [12]:
print('Processing unknown faces...')
# Now let's loop over a folder of faces we want to label
for filename in os.listdir(UNKNOWN_FACES_DIR):

    # Load image
    print(f'Filename {filename}', end='')
    image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}')

    # This time we first grab face locations - we'll need them to draw boxes
    locations = face_recognition.face_locations(image, model="cnn")

    # Now since we know loctions, we can pass them to face_encodings as second argument
    # Without that it will search for faces once again slowing down whole process
    encodings = face_recognition.face_encodings(image, locations)


Processing unknown faces...
Filename IMG_20200407_174629_063.jpgFilename WhatsApp Image 2020-04-11 at 10.08.33 AM (1).jpegFilename WhatsApp Image 2020-04-11 at 10.08.33 AM.jpeg

In [13]:
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # But this time we assume that there might be more faces in an image - we can find faces of dirrerent people
    print(f', found {len(encodings)} face(s)')
    for face_encoding, face_location in zip(encodings, locations):

        # We use compare_faces (but might use face_distance as well)
        # Returns array of True/False values in order of passed known_faces
        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)

        # Since order is being preserved, we check if any face was found then grab index
        # then label (name) of first matching known face withing a tolerance
        match = None
        if True in results:  # If at least one is true, get a name of first of found labels
            match = known_names[results.index(True)]
            print(f' - {match} from {results}')

            # Each location contains positions in order: top, right, bottom, left
            top_left = (face_location[3], face_location[0])
            bottom_right = (face_location[1], face_location[2])

            # Get color by name using our fancy function
            color = name_to_color(match)

            # Paint frame
            cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)

            # Now we need smaller, filled grame below for a name
            # This time we use bottom in both corners - to start from bottom and move 50 pixels down
            top_left = (face_location[3], face_location[2])
            bottom_right = (face_location[1], face_location[2] + 22)

            # Paint frame
            cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)

            # Wite a name
            cv2.putText(image, match, (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), FONT_THICKNESS)

    # Show image
    cv2.imshow(filename, image)
    cv2.waitKey(0)
    cv2.destroyWindow(filename)

, found 1 face(s)


ValueError: operands could not be broadcast together with shapes (11,) (128,) 